In [1]:
import os
import time
import numpy as np
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

#　データフレーム　投手ステータス用

#関数
def league_chg(league): # 0: セ・リーグ 1: パ・リーグ
    if league == 0:
        return team_ini_c
    elif league == 1:
        return team_ini_p
    else:
        print("0 or 1を入力")


def select_year(year):
    return year


def get_soup(year, team):
    url = f"http://www.baseless.org/data/source/{year}/rnk_D_SPEED_{team}.html"
    # url = f"http://www.baseless.org/data/source/{year}/dat_{team}.html"
    html = urlopen(url)
    soup = BeautifulSoup(html,"html5lib")
    return soup


def get_soup_num(year, team):
    # url = f"http://www.baseless.org/data/source/{year}/rnk_D_SPEED_{team}.html"
    url = f"http://www.baseless.org/data/source/{year}/dat_{team}.html"
    html = urlopen(url)
    soup = BeautifulSoup(html,"html5lib")
    return soup

#初期設定用
#保存用フォルダ作成
comp_dir = "comp_files"
try:
    os.mkdir(comp_dir)
except:
    print("フォルダは既に存在しています")

team_ini_c = ["S", "T", "G", "C", "D", "BA"] #セ・リーグ
team_ini_p = ["OBU", "M", "E", "H", "F", "L"] #パ・リーグ
year_list = ["2011", "2012", "2013", "2014", "2016", "2018", "2020", "2022"]
num = len(year_list)
#dfカラム用
pit_col = ["ランク","球団", "選手名", "球速", "コントロール", "スタミナ",
    "総変化量", "スライダー系", "カーブ系", "フォーク系", "シンカー系", "シュート系"]
#リーグ選択
team = league_chg(0)    # 0:セ 1:パ

#---------------------------------------------------------------------------------------------------------------------
#main処理
for m in range(num):
    year = select_year(year_list[m]) #年代指定　2016, 2018, 2020, 2022
    df_list = []    #各球団のステータスのDataFrame格納用
    #　データフレーム　ステータス取得用
    print("START", year, "status")
    for i in range(6):
        sc_data = []
        soup = get_soup(year, team[i])
        table = soup.findAll("table")
        for row in table:
            sc_data.append(row.get_text(",", strip=True))

        for i in sc_data:
            sc = i.split(",")
        sc_list = sc[6:]    #最初の6個不要
        sc_array = np.array(sc_list) # sc_list.shape (420,)
        # データ成形 列:12
        sc_array = sc_array.reshape(int(len(sc_array)/12) ,12)
        #df作成
        sc_df = pd.DataFrame(sc_array, columns=pit_col)
        sc_df = sc_df.drop("ランク", axis=1)
        sc_df["選手名"] = sc_df["選手名"].str.replace("��", "高").str.replace("�ｱ", "崎").str.replace("�縺@", "桑")
        df_list.append(sc_df)   #dfをリストへ格納
        time.sleep(1)
    print("END", year, "status")
    print("--------------------------------")

    #　データフレーム　背番号ほか用　設定変更で特殊能力など取得可
    print("START", year, "number")
    num_df_list = []    #各球団の背番号などのDataFrame格納用
    for i in range(6): #for i in range(len(league_chg(1)))
        p_ab_scray = []
        soup = get_soup_num(year, team[i])
        table = soup.findAll("b", {"class": "if w"}) #"nm"ほかで各要素
        for row in table:
            # print(row.get_text(",", strip=True))
            p_ab_scray.append(row.get_text(",", strip=True))
        #スクレイピング　奥川', '11', 'オーバースロー16'・・・・　野手要素も取れている　投手だと削除
        info_list = []
        for i in range(len(p_ab_scray)):
            info_list.append(p_ab_scray[i].split(","))
        num_df = pd.DataFrame(info_list)
        num_df = num_df[[0,1]] #0,1列目のみ利用
        num_df.columns = ["選手名", "背番号"]
        num_df["選手名"] = num_df["選手名"].str.replace("��", "高").str.replace("�ｱ", "崎").str.replace("�縺@", "桑")

        num_df_list.append(num_df)  #dfをリストへ格納
        time.sleep(1)
    print("END", year, "number")
    print("--------------------------------")

    #df 結合
    re_col = ["選手名", "球速", "コントロール", "スタミナ",
    "総変化量", "スライダー系", "カーブ系", "フォーク系", "シンカー系", "シュート系","球団", "背番号"]
    comp_p_df = [] #リストに格納して、concatで一括処理
    for i in range(6):
        merge_sc = pd.merge(df_list[i], num_df_list[i], on="選手名", how='outer')
        merge_sc = merge_sc.reindex(re_col, axis='columns')
        comp_df = merge_sc.dropna() #num_dfに野手がいるため NaNの部分を削除
        comp_p_df.append(comp_df)

    #comp_p_dfを一体化
    pitcher_databese = pd.concat(comp_p_df,ignore_index = True, axis=0)
    pitcher_databese["球団"] = pitcher_databese["球団"].str.replace("横浜", "DeNA")
    #最終データ保存
    if team[0] == "OBU":
        pitcher_databese.to_csv(f"./comp_files/{str(year)}_plg_Pitchers.csv", encoding="UTF-8", index=False)
    else:
        pitcher_databese.to_csv(f"./comp_files/{str(year)}_clg_Pitchers.csv", encoding="UTF-8", index=False)

    print(year,"年度ピッチャーDB用データ保存")
    print("--------------------------------")

    #CSV個別保存 任意 年度ごとのチーム別データ
    # for i in range(6):
    #     if team[0] == "OBU":
    #         comp_p_df[i].to_csv(f"./comp_files/{str(year)}_plg_indi_pitchers_{team[i]}.csv", encoding="UTF-8")
    #     else:
    #         comp_p_df[i].to_csv(f"./comp_files/{str(year)}_clg_indi_pitchers_{team[i]}.csv", encoding="UTF-8")

print("全年度取得完了")
print("--------------------------------")

START 2011 status
END 2011 status
--------------------------------
START 2011 number
END 2011 number
--------------------------------
2011 年度ピッチャーDB用データ保存
--------------------------------
START 2012 status
END 2012 status
--------------------------------
START 2012 number
END 2012 number
--------------------------------
2012 年度ピッチャーDB用データ保存
--------------------------------
START 2013 status
END 2013 status
--------------------------------
START 2013 number
END 2013 number
--------------------------------
2013 年度ピッチャーDB用データ保存
--------------------------------
START 2014 status
END 2014 status
--------------------------------
START 2014 number
END 2014 number
--------------------------------
2014 年度ピッチャーDB用データ保存
--------------------------------
START 2016 status
END 2016 status
--------------------------------
START 2016 number
END 2016 number
--------------------------------
2016 年度ピッチャーDB用データ保存
--------------------------------
START 2018 status
END 2018 status
--------------------------